**Install Dependencies**

In [ ]:
!pip install -q flask pyngrok transformers accelerate bitsandbytes sentence-transformers faiss-cpu PyPDF2

**Create app.py**

In [ ]:
%%writefile app.py
# ==============================================
# 📚 Intelligent Academic Research Assistant
# Flask + RAG (PDFs + FAISS + Mistral-7B)
# ==============================================

import os, gc, torch, faiss, numpy as np, re, logging, warnings, io as py_io
from flask import Flask, render_template, request
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from functools import lru_cache
from transformers.utils import logging as hf_logging

# ------------------------------------------------
# ⚙️ Suppress noisy logs
# ------------------------------------------------
hf_logging.set_verbosity_error()
logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

app = Flask(__name__)

# ==============================================
# 🧠 STEP 3 — LAZY-LOADED MODELS (CACHED)
# (same models as your original code)
# ==============================================
@lru_cache(maxsize=1)
def get_embed_model():
    print("🔄 Loading embedding model (all-MiniLM-L12-v2)...")
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
    print("✅ Embedding model ready and cached.")
    return model

@lru_cache(maxsize=1)
def get_llm_pipeline():
    print("🔄 Loading Mistral 7B Instruct (4-bit)...")
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        load_in_4bit=True
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    print("✅ Mistral 7B Instruct model loaded and cached.")
    return pipe


# ==============================================
# 📂 GLOBAL STATE FOR RAG (PDFs, Chunks, Index)
# ==============================================
chunk_texts = []        # text chunks
chunk_sources = []      # source PDF for each chunk
index = None            # FAISS index
pdf_files = []          # uploaded PDF filenames


# ==============================================
# 📂 STEP 4 — UPLOAD & EXTRACT PDF TEXT (Helper)
# Same logic as your original, but wrapped for Flask
# ==============================================
def process_uploaded_pdfs(file_storages):
    """
    Reads uploaded PDF files, extracts text, creates chunks,
    and builds global chunk_texts, chunk_sources and FAISS index.
    """
    global chunk_texts, chunk_sources, index, pdf_files

    pdf_files = [f.filename for f in file_storages if f and f.filename.lower().endswith(".pdf")]
    if not pdf_files:
        raise ValueError("❌ No valid PDF files uploaded.")

    print(f"✅ Uploaded {len(pdf_files)} file(s): {pdf_files}")

    # For size info (not required for RAG)
    pdf_texts = ""
    for file in file_storages:
        if not file or not file.filename.lower().endswith(".pdf"):
            continue
        reader = PdfReader(py_io.BytesIO(file.read()))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
        pdf_texts += f"\n\n--- From {file.filename} ---\n\n{text}"

    print(f"✅ Extracted {len(pdf_texts)//1000}K characters from all PDFs.")

    # Rebuild chunks and sources
    chunk_texts = []
    chunk_sources = []

    for file in file_storages:
        if not file or not file.filename.lower().endswith(".pdf"):
            continue
        file.stream.seek(0)
        reader = PdfReader(py_io.BytesIO(file.read()))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
        words = text.split()
        # Same chunking as your original: chunk_size=500, overlap=50
        for i in range(0, len(words), 500 - 50):
            chunk = " ".join(words[i:i + 500])
            chunk_texts.append(chunk)
            chunk_sources.append(file.filename)

    print(f"✅ Created {len(chunk_texts)} chunks with source mapping.")

    # Build FAISS index
    build_faiss_index()


# ==============================================
# 🧩 STEP 6 — BUILD FAISS INDEX
# (same idea as your original get_faiss_index)
# ==============================================
def get_faiss_index(embeddings):
    print("🔄 Building FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.astype("float32"))
    print("✅ FAISS index built successfully.")
    return index

def build_faiss_index():
    global index
    if not chunk_texts:
        raise ValueError("❌ No chunks available to build FAISS index.")

    embed_model = get_embed_model()
    embeddings = embed_model.encode(chunk_texts, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")
    index = get_faiss_index(embeddings)


# ==============================================
# 🧠 DYNAMIC RELEVANCE FILTER (same logic)
# ==============================================
def filter_relevant_chunks(user_query, retrieved_chunks):
    """
    Dynamically filters retrieved chunks using query-derived keywords.
    Ensures only topically relevant text is used for generation.
    """
    q = user_query.lower()
    stopwords = {
        "what", "is", "are", "in", "the", "of", "and", "for", "to",
        "explain", "describe", "how", "does", "do", "difference", "between"
    }
    query_keywords = [w for w in re.findall(r"\w+", q) if w not in stopwords and len(w) > 2]

    scored_chunks = []
    for c in retrieved_chunks:
        score = sum(k in c.lower() for k in query_keywords)
        scored_chunks.append((score, c))

    scored_chunks.sort(reverse=True, key=lambda x: x[0])
    relevant_chunks = [c for s, c in scored_chunks if s > 0]

    # fallback if nothing matches
    if len(relevant_chunks) < 1:
        relevant_chunks = [c for _, c in scored_chunks[:3]]

    return "\n\n".join(relevant_chunks)


# ==============================================
# 🧠 RAG ANSWER MODE — Full Academic Response
# Based on your original rag_query()
# ==============================================
def rag_answer(user_query, top_k=3):
    """
    Returns full academic RAG answer + sources for a given query.
    """
    if not user_query:
        return "⚠️ No question entered.", ""

    if index is None or not chunk_texts:
        return "⚠️ No documents indexed yet. Please upload PDFs first.", ""

    embed_model = get_embed_model()
    generator = get_llm_pipeline()

    query_emb = embed_model.encode([user_query])
    query_emb = np.array(query_emb).astype("float32")
    D, I = index.search(query_emb, k=top_k)

    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    retrieved_sources = [chunk_sources[i] for i in I[0]]

    # ✅ Apply dynamic filter
    context = filter_relevant_chunks(user_query, retrieved_chunks)
    sources_used = ", ".join(sorted(set(retrieved_sources)))

    prompt = f"""
You are an academic research assistant.
Use ONLY the context provided below to answer factually and in an academic tone.

Context:
{context}

Question:
{user_query}

Answer:
"""

    import sys
    _stderr = sys.stderr
    sys.stderr = py_io.StringIO()

    response_raw = generator(
        prompt,
        max_new_tokens=450,
        temperature=0.3,
        top_p=0.9,
        do_sample=False
    )[0]["generated_text"]

    sys.stderr = _stderr
    response_clean = response_raw.split("Answer:")[-1].strip() if "Answer:" in response_raw else response_raw.strip()

    # Optional log (same idea as original)
    try:
        with open("rag_log.txt", "a", encoding="utf-8") as log:
            log.write(f"Question: {user_query}\nAnswer: {response_clean}\nSources: {sources_used}\n{'='*100}\n\n")
    except Exception as e:
        print(f"⚠️ Logging error: {e}")

    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass

    return response_clean, sources_used


# ==============================================
# 🩵 RAG SUMMARIZATION MODE — Concise Summary
# Based on your original rag_summary()
# ==============================================
def rag_summary_answer(user_query, top_k=3):
    """
    Returns concise academic summary + sources for a given query.
    """
    if not user_query:
        return "⚠️ No question entered.", ""

    if index is None or not chunk_texts:
        return "⚠️ No documents indexed yet. Please upload PDFs first.", ""

    embed_model = get_embed_model()
    generator = get_llm_pipeline()

    query_emb = embed_model.encode([user_query])
    query_emb = np.array(query_emb).astype("float32")
    D, I = index.search(query_emb, k=top_k)

    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    retrieved_sources = [chunk_sources[i] for i in I[0]]

    # ✅ Apply dynamic filter
    context = filter_relevant_chunks(user_query, retrieved_chunks)
    sources_used = ", ".join(sorted(set(retrieved_sources)))

    prompt = f"""
You are an academic summarizer.
Use ONLY the context below to generate a concise and factual summary that directly answers the user's question.
If the text lists multiple items, challenges, or methods, combine or enumerate them clearly.
Provide an academically concise summary (4–6 sentences).

Context:
{context[:4000]}

Question:
{user_query}

Summary:
"""

    response = generator(
        prompt,
        max_new_tokens=220,
        temperature=0.25,
        top_p=0.9,
        do_sample=False
    )[0]["generated_text"]

    if "Summary:" in response:
        summary_clean = response.split("Summary:")[-1].strip()
    else:
        summary_clean = response.strip()

    summary_clean = summary_clean.replace("in clear academic paragraphs:", "").strip()
    if not summary_clean.endswith(('.', '!', '?')):
        summary_clean = summary_clean.rstrip(",;:") + "."

    # Optional log
    try:
        with open("rag_summary_log.txt", "a", encoding="utf-8") as log:
            log.write(f"Question: {user_query}\nSummary: {summary_clean}\nSources: {sources_used}\n{'='*100}\n\n")
    except Exception as e:
        print(f"⚠️ Logging error: {e}")

    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass

    return summary_clean, sources_used


# ==============================================
# 🌐 MAIN FLASK ROUTE
# Upload PDFs once, then ask multiple questions
# ==============================================
@app.route("/", methods=["GET", "POST"])
def home():
    global index, chunk_texts, chunk_sources, pdf_files

    answer = ""
    summary = ""
    sources_answer = ""
    sources_summary = ""
    status_msg = ""
    error = ""
    mode = "answer"  # default radio selection

    has_index = index is not None and len(chunk_texts) > 0

    if request.method == "POST":
        action = request.form.get("action")

        # ---------------------------------
        # 📂 ACTION: Upload and Index PDFs
        # ---------------------------------
        if action == "upload":
            uploaded_files = request.files.getlist("pdfs")
            if not uploaded_files or all(f.filename == "" for f in uploaded_files):
                error = "⚠️ Please upload at least one academic PDF."
            else:
                try:
                    process_uploaded_pdfs(uploaded_files)
                    has_index = True
                    status_msg = f"✅ Indexed {len(chunk_texts)} chunks from {len(pdf_files)} PDF(s)."
                except Exception as e:
                    error = f"⚠️ Error while processing PDFs: {e}"

        # ---------------------------------
        # 🔍 ACTION: Ask Question (Answer/Summary)
        # ---------------------------------
        elif action == "query":
            user_query = request.form.get("query", "").strip()
            mode = request.form.get("mode", "answer")

            if not user_query:
                error = "⚠️ Please enter an academic question."
            elif index is None or not chunk_texts:
                error = "⚠️ Please upload and index PDFs before asking questions."
            else:
                if mode == "answer":
                    answer, sources_answer = rag_answer(user_query)
                elif mode == "summary":
                    summary, sources_summary = rag_summary_answer(user_query)
                else:
                    error = "⚠️ Unknown mode selected."

    return render_template(
        "index.html",
        has_index=has_index,
        pdf_files=pdf_files,
        status_msg=status_msg,
        error=error,
        answer=answer,
        summary=summary,
        sources_answer=sources_answer,
        sources_summary=sources_summary,
        mode=mode,
    )


# ==============================================
# 🚀 RUN FLASK APP
# ==============================================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

**Create Folders**

In [ ]:
!mkdir -p templates
!mkdir -p static


**templates/index.html**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>📚 Intelligent Academic Research Assistant</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">
            <h1>📚 Intelligent Academic Research Assistant</h1>
            <p>Upload academic PDFs once, then ask research questions with Mistral + FAISS powered RAG.</p>

            <!-- PDF Upload Form -->
            <form method="post" enctype="multipart/form-data" class="card">
                <input type="hidden" name="action" value="upload" />
                <label class="file-label">
                    📂 Upload academic PDF(s)
                    <input type="file" name="pdfs" accept=".pdf" multiple required />
                </label>
                <button type="submit" class="btn-primary">Index PDFs 📑</button>
            </form>

            {% if pdf_files %}
            <div class="status-box">
                <p><strong>📁 Uploaded PDFs:</strong></p>
                <ul>
                    {% for f in pdf_files %}
                    <li>{{ f }}</li>
                    {% endfor %}
                </ul>
            </div>
            {% endif %}

            {% if status_msg %}
            <p class="status-msg">{{ status_msg }}</p>
            {% endif %}

            {% if error %}
            <p class="error-msg">{{ error }}</p>
            {% endif %}

            <!-- Query Form -->
            <form method="post" class="card">
                <input type="hidden" name="action" value="query" />
                <textarea name="query" rows="4" placeholder="Enter your academic research question..." required></textarea>

                <div class="mode-selector">
                    <label>
                        <input type="radio" name="mode" value="answer" {% if mode == "answer" %}checked{% endif %} />
                        🧠 Full Academic Answer
                    </label>
                    <label>
                        <input type="radio" name="mode" value="summary" {% if mode == "summary" %}checked{% endif %} />
                        🩵 Concise Academic Summary
                    </label>
                </div>

                <button type="submit" class="btn-primary">Generate Response 🚀</button>
            </form>
        </div>
    </div>

    <!-- Results Section -->
    {% if answer or summary %}
    <div class="result-section fade-in">
        <div class="result-card">
            {% if answer %}
            <h2>🧠 Full Academic Answer</h2>
            <p class="result-text">{{ answer }}</p>
            {% if sources_answer %}
            <p class="sources">📚 Sources used: {{ sources_answer }}</p>
            {% endif %}
            {% endif %}

            {% if summary %}
            <h2>🩵 Concise Academic Summary</h2>
            <p class="result-text">{{ summary }}</p>
            {% if sources_summary %}
            <p class="sources">📚 Sources used: {{ sources_summary }}</p>
            {% endif %}
            {% endif %}
        </div>
    </div>
    {% endif %}
</body>
</html>


**static/style.css**

In [ ]:
%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    margin: 0;
    font-family: 'Poppins', sans-serif;
    color: #fff;
}

/* Background */
.hero-section {
    position: relative;
    min-height: 100vh;
    display: flex;
    align-items: center;
    padding: 40px 8%;
    background: linear-gradient(135deg, #141e30, #243b55);
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0,0,0,0.35);
}

/* Main card */
.hero-content {
    position: relative;
    z-index: 2;
    max-width: 720px;
    background: rgba(255,255,255,0.08);
    border-radius: 24px;
    padding: 28px 32px;
    backdrop-filter: blur(8px);
    box-shadow: 0 10px 30px rgba(0,0,0,0.45);
}

h1 {
    margin: 0 0 10px;
    font-size: 2rem;
    color: #ffdf9e;
}

p {
    margin: 4px 0;
}

/* Cards */
.card {
    margin-top: 18px;
    padding: 14px 16px;
    background: rgba(0,0,0,0.35);
    border-radius: 16px;
    display: flex;
    flex-direction: column;
    gap: 10px;
}

/* File input */
.file-label {
    font-size: 0.9rem;
    background: rgba(255,255,255,0.15);
    padding: 10px 12px;
    border-radius: 10px;
}

.file-label input {
    display: block;
    margin-top: 8px;
}

/* Textarea */
textarea {
    width: 100%;
    padding: 10px;
    border-radius: 10px;
    border: none;
    resize: vertical;
    min-height: 100px;
    background: rgba(255,255,255,0.95);
    color: #111;
    font-size: 0.95rem;
}

/* Buttons */
.btn-primary {
    padding: 10px 12px;
    border: none;
    border-radius: 10px;
    background: linear-gradient(135deg, #00c6ff, #0072ff);
    color: #fff;
    font-weight: 600;
    cursor: pointer;
    font-size: 0.95rem;
    align-self: flex-start;
    transition: 0.25s ease;
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 16px rgba(0,0,0,0.35);
}

/* Status / error */
.status-box {
    margin-top: 12px;
    padding: 10px 12px;
    background: rgba(0,0,0,0.35);
    border-radius: 10px;
    font-size: 0.9rem;
}

.status-msg {
    margin-top: 8px;
    color: #b3ffd1;
    font-size: 0.9rem;
}

.error-msg {
    margin-top: 8px;
    color: #ffb3b3;
    font-size: 0.9rem;
}

/* Mode selector */
.mode-selector {
    display: flex;
    gap: 16px;
    font-size: 0.9rem;
}

.mode-selector label {
    display: flex;
    align-items: center;
    gap: 6px;
}

/* Results */
.result-section {
    background: #0d111f;
    padding: 40px 20px;
    display: flex;
    justify-content: center;
}

.result-card {
    max-width: 900px;
    width: 100%;
    background: rgba(255,255,255,0.05);
    padding: 24px 28px;
    border-radius: 16px;
    color: #e2e5ff;
}

.result-card h2 {
    margin-top: 0;
}

.result-text {
    white-space: pre-wrap;
    line-height: 1.5;
    margin-bottom: 10px;
}

.sources {
    font-size: 0.85rem;
    color: #a9b8ff;
}

/* Animation */
.fade-in {
    animation: fadeInUp 0.6s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(18px); }
    to   { opacity: 1; transform: translateY(0); }
}


**Run Flask + ngrok**

In [ ]:
# Kill existing processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
# (Optional) If a specific PID is blocking:
!kill -9 617

In [ ]:
# Start Flask in background
!nohup python app.py > flask.log 2>&1 &

In [ ]:

# Start ngrok
from pyngrok import ngrok, conf

# Enter your NGROK auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)
